In [16]:
import numpy as np
import pandas as pd
# Dados séries temporai
from datetime import datetime, timedelta
import pytz
# Integração com MetaTrader 5
import datetime as dt
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")


In [17]:
mt5.initialize()

True

In [18]:
def baixar_dados(ticker):
    cotacoes_mt5_all = []

    timezone = pytz.timezone("America/Sao_Paulo")
    data_inicio = datetime.now() - timedelta(days=600)
    data_fim = datetime.now()

    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_H1, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)
    return pd.concat(cotacoes_mt5_all, axis=0)


In [19]:
def tratamento_df(df_final):
    df_final = df_final.drop('time', axis =1)

    df_final = df_final.reset_index()

    # Criar a coluna "data" contendo apenas a parte da data
    df_final['data'] = df_final['time'].dt.date

    # Criar a coluna "hora" contendo apenas a parte da hora
    df_final['hora'] = df_final['time'].dt.time

    df_final = df_final.drop('time', axis =1)
    return df_final

In [20]:
def window_func(df_final):
    # Sort the DataFrame by 'data' and 'ticker', then by 'hora'
    df_final.sort_values(by=['data', 'ticker', 'hora'], inplace=True)

    # Apply the window function and create a new column 'numeration'
    df_final['numeration'] = df_final.groupby(['data', 'ticker']).cumcount() + 1
        
    # Find the maximum 'numeration' for each day and ticker
    max_numeration_per_day = df_final.groupby(['data', 'ticker'])['numeration'].max()

    # Merge the result back into the original DataFrame to get the closing value (close) for each day
    df_final = df_final.merge(max_numeration_per_day, on=['data', 'ticker'], suffixes=('', '_max'))

    closing_values_per_day = df_final[df_final['numeration'] == df_final['numeration_max']][['data', 'ticker', 'close']]

    df_final = df_final.merge(closing_values_per_day, on=['data', 'ticker'], how='left')

    df_final.rename(columns={'close_y': 'fechamento_do_dia'}, inplace=True)
    #df_final.drop(['numeration','numeration_max'], inplace=True, axis=1)
    return df_final

In [21]:
def dia_anterior(df):
    df_anterior = df[['data', 'fechamento_do_dia']].groupby('data').last().reset_index()
    df_anterior.columns = ['data', 'fechamento_dia_anterior']

    df_anterior['fechamento_dia_anterior'] = df_anterior['fechamento_dia_anterior'].shift(1)

    # Faça um left join com base na coluna 'data'
    df_resultado = pd.merge(df, df_anterior, on='data', how='left')
    return df_resultado

In [22]:
def create_perc(df_ativo, percentual):
    #Ajustar variáveis
    df_ativo['entrada'] = df_ativo['fechamento_dia_anterior'] - (df_ativo['fechamento_dia_anterior'] * percentual)
    
    df_ativo['percentual'] = percentual
    return df_ativo

In [23]:
def create_stats (df_ativo, dias_analisados, horario):
    df_ativo['numeration'] = df_ativo['numeration'].astype(int)
    df_ativo = df_ativo.loc[df_ativo['numeration'] == horario]

    df_ativo['tem_entrada'] = np.where(df_ativo['open'] <= df_ativo['entrada'],1,0)
    df_ativo['alvo'] = np.where(df_ativo['tem_entrada'] == 1 , (df_ativo['fechamento_do_dia'] / df_ativo['open'] -1) *100, 0) 

    df_filtrado = df_ativo.loc[df_ativo['tem_entrada']==1]
    df_filtrado = df_filtrado.tail(dias_analisados)
    df_filtrado['alvo_acumulado'] = df_filtrado['alvo'].cumsum()
    return df_filtrado

In [24]:
def calcular_drawdown(df_filtrado, horario):
    df_filtrado['max_acumulado'] = df_filtrado['alvo_acumulado'].cummax()
    df_filtrado['drawdown'] = df_filtrado['alvo_acumulado'] - df_filtrado['max_acumulado']

    # Calcular o drawdown máximo
    drawdown_maximo = df_filtrado['drawdown'].min()

    # Localizar o índice do valor mínimo em 'drawdown'
    indice_drawdown_maximo = df_filtrado['drawdown'].idxmin()

    # Acessar a data correspondente ao índice do drawdown máximo
    data_drawdown_maximo = df_filtrado.loc[indice_drawdown_maximo, 'data']

    # Calcular a taxa de acerto
    resultado = df_filtrado['alvo'] > 0
    taxa_acerto = sum(resultado) / len(df_filtrado['alvo'])

    # Separar resultados positivos e negativos
    resultados_positivos = df_filtrado.loc[df_filtrado['alvo'] > 0, 'alvo']
    resultados_negativos = df_filtrado.loc[df_filtrado['alvo'] <= 0, 'alvo']

    try:
        pior_dia = min(resultados_negativos)
        melhor_dia = max(resultados_positivos)
    except:
        pior_dia = 0
        melhor_dia = 0
        
    indice_pior_dia = df_filtrado['alvo'].idxmin()
    data_pior_dia = df_filtrado.loc[indice_pior_dia, 'data']
    
    quantidade_trades = len(resultados_positivos) + len(resultados_negativos)
    # Calcular médias
    media_positivos = resultados_positivos.mean()
    media_negativos = resultados_negativos.mean()
    media_geral = df_filtrado['alvo'].mean()

    # Calcular frequência de resultados positivos e negativos
    freq_positivos = len(resultados_positivos) / len(df_filtrado)
    freq_negativos = len(resultados_negativos) / len(df_filtrado)

    # Calcular expectativa matemática
    expectativa = freq_positivos * media_positivos - freq_negativos * abs(media_negativos)

    # Calcular retorno médio diário
    retorno_medio_diario = df_filtrado['alvo'].mean()

    # Calcular desvio padrão diário
    desvio_padrao_diario = df_filtrado['alvo'].std()

    # Calcular índice de Sharpe
    indice_sharpe = np.sqrt(252) * retorno_medio_diario / desvio_padrao_diario
    
    ativo =  df_filtrado.iloc[2]['ticker']
    retorno_acumulado = df_filtrado.iloc[-1]['alvo_acumulado']
    primeiro_dia = df_filtrado.iloc[1]['data']
    
    resultado_dict = {
        'ativo': ativo,
        'horário': horario,
        'Retorno Acumulado': retorno_acumulado,
        'Melhor dia': melhor_dia,           
        'Pior Dia':pior_dia, 
        'Data Pior dia': data_pior_dia,
        'Primeiro Dia': primeiro_dia,
        'Quantidade de Trades': quantidade_trades,
        'Taxa de Acerto': taxa_acerto,
        'Media Geral': media_geral,
        'Média Positivos': media_positivos,
        'Média Negativos': media_negativos,
        'Expectativa Matemática': expectativa,
        'Índice de Sharpe': indice_sharpe,
        'Drawdown Máximo': drawdown_maximo,
        'Data do Drawdown': data_drawdown_maximo
    }

    return resultado_dict

In [25]:
ibov = ['SEQL3',
'AERI3',
'WEST3',
'TRAD3',
'KRSA3',
'ESPA3',
'IFCM3',
'NGRD3',
'VIIA3',
'MEAL3',
'SGPS3',
'BMGB4',
'LPSB3',
'PDTC3',
'SHOW3',
'CVCB3',
'LUPA3',
'MGLU3',
'UCAS3',
'BRIT3',
'MLAS3',
'COGN3',
'TECN3',
'PINE4',
'NINJ3',
'TCSA3',
'MBLY3',
'HBSA3',
'RAIZ4',
'CIEL3',
'HBOR3',
'CMIN3',
'SYNE3',
'PGMN3',
'QUAL3',
'PFRM3',
'ZAMP3',
'FIQE3',
'OSXB3',
'CSED3',
'POMO3',
'ANIM3',
'HAPV3',
'DMVF3',
'MELK3',
'AGXY3',
'ALPK3',
'CBAV3',
'CLSA3',
'OPCT3',
'PDGR3',
'LVTC3',
'JHSF3',
'RDNI3',
'POMO4',
'HBRE3',
'GFSA3',
'CEAB3',
'SEER3',
'TRIS3',
'LJQQ3',
'WIZC3',
'GUAR3',
'ALLD3',
'AVLL3',
'PETZ3',
'USIM5',
'LWSA3',
'USIM3',
'GRND3',
'MTRE3',
'CAML3',
'GMAT3',
'BOAS3',
'EVEN3',
'MRFG3',
'JALL3',
'PTBL3',
'ECOR3',
'CPLE6',
'BPAN4',
'POSI3',
'DEXP4',
'LAVV3',
'ALPA4',
'DXCO3',
'GOLL4',
'CASH3',
'DEXP3',
'ITSA4',
'ITSA3',
'STBP3',
'RAPT3',
'KEPL3',
'ALPA3',
'BRFS3',
'JSLG3',
'MATD3',
'BEEF3',
'CXSE3',
'RANI3',
'SIMH3',
'PLPL3',
'SOMA3',
'AESB3',
'MEGA3',
'MDNE3',
'ONCO3',
'BMEB4',
'SOJA3',
'CRFB3',
'VITT3',
'VAMO3',
'PRNR3',
'CEDO4',
'MOVI3',
'CMIG4',
'ODPV3',
'RAPT4',
'TTEN3',
'MILS3',
'FHER3',
'ENEV3',
'CCRO3',
'PNVL3',
'SBFG3',
'GOAU3',
'EUCA4',
'BMEB3',
'PORT3',
'TEND3',
'GOAU4',
'FRAS3',
'DASA3',
'ASAI3',
'AURE3',
'MRVE3',
'BMOB3',
'BBDC3',
'TFCO4',
'BRSR3',
'ARML3',
'ENAT3',
'ROMI3',
'BRSR6',
'DESK3',
'TIMS3',
'MYPK3',
'B3SA3',
'ELMD3',
'CSUD3',
'BBDC4',
'TASA4',
'TASA3',
'FLRY3',
'BRBI11',
'VLID3',
'VBBR3',
'AZUL4',
'EUCA3',
'CURY3',
'EMBR3',
'NTCO3',
'CSMG3',
'GGPS3',
'CMIG3',
'UGPA3',
'NEOE3',
'ABCB4',
'LREN3',
'CSAN3',
'JBSS3',
'PCAR3',
'LOGG3',
'VVEO3',
'VULC3',
'SAPR11',
'BLAU3',
'YDUQ3',
'BRAP3',
'DIRR3',
'IGTI11',
'AMBP3',
'RECV3',
'EZTC3',
'RAIL3',
'BRAP4',
'KLBN11',
'BRKM5',
'INTB3',
'SCAR3',
'SMFT3',
'ALSO3',
'TGMA3',
'AALR3',
'ITUB3',
'BRKM3',
'OFSA3',
'CYRE3',
'LAND3',
'TRPL4',
'GGBR3',
'AGRO3',
'SQIA3',
'MULT3',
'PSSA3',
'RADL3',
'ITUB4',
'GGBR4',
'TUPY3',
'ALUP11',
'TOTS3',
'PETR4',
'VIVA3',
'APER3',
'BBSE3',
'EQTL3',
'PETR3',
'BPAC11',
'SMTO3',
'TRPL3',
'RDOR3',
'CPFE3',
'ELET3',
'TAEE11',
'RRRP3',
'ORVR3',
'IRBR3',
'WEGE3',
'ELET6',
'MDIA3',
'EGIE3',
'HYPE3',
'SLCE3',
'LEVE3',
'BBAS3',
'PRIO3',
'LOGN3',
'ENGI11',
'SUZB3',
'FESA4',
'SBSP3',
'CLSC3',
'CLSC4',
'RENT3',
'VALE3',
'ARZZ3']

In [26]:
horarios = [1,2,3,4,5,6,7,8,9]
lista_resultados = []
for ativo in ibov:
    print( f'Baixando os dados de {ativo}')
    for hora in horarios:
        try:
            df_final = baixar_dados(ativo)
            df_final = tratamento_df(df_final)
            df_final = window_func(df_final)
            df_final = dia_anterior(df_final)
            df_final = create_perc(df_final, 0.015)
            df_final = create_stats(df_final, dias_analisados = 30,horario = hora)
            resultado = calcular_drawdown(df_final, hora)
            lista_resultados.append(resultado)
        except:
            continue

df_resultado = pd.DataFrame(lista_resultados)

Baixando os dados de SEQL3
Baixando os dados de AERI3
Baixando os dados de WEST3
Baixando os dados de TRAD3
Baixando os dados de KRSA3
Baixando os dados de ESPA3
Baixando os dados de IFCM3
Baixando os dados de NGRD3
Baixando os dados de VIIA3
Baixando os dados de MEAL3
Baixando os dados de SGPS3
Baixando os dados de BMGB4
Baixando os dados de LPSB3
Baixando os dados de PDTC3
Baixando os dados de SHOW3
Baixando os dados de CVCB3
Baixando os dados de LUPA3
Baixando os dados de MGLU3
Baixando os dados de UCAS3
Baixando os dados de BRIT3
Baixando os dados de MLAS3
Baixando os dados de COGN3
Baixando os dados de TECN3
Baixando os dados de PINE4
Baixando os dados de NINJ3
Baixando os dados de TCSA3
Baixando os dados de MBLY3
Baixando os dados de HBSA3
Baixando os dados de RAIZ4
Baixando os dados de CIEL3
Baixando os dados de HBOR3
Baixando os dados de CMIN3
Baixando os dados de SYNE3
Baixando os dados de PGMN3
Baixando os dados de QUAL3
Baixando os dados de PFRM3
Baixando os dados de ZAMP3
B

In [27]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] >= 0.75) 
                    & (df_resultado['Pior Dia'] > -1.5) 
                    & (df_resultado['Media Geral'] > 0.3)
                    & (df_resultado['Quantidade de Trades'] >= 20)
                    ].sort_values(['Taxa de Acerto'], ascending=False)
selecionados

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
1463,RAIL3,8,12.688098,2.282609,-0.962696,2022-12-13,2022-11-16,29,0.827586,0.437521,0.624352,-0.459270,0.437521,10.254318,-1.206360,2023-01-03
336,DMVF3,1,40.952225,4.807692,-0.928793,2023-04-10,2022-11-30,30,0.800000,1.365074,1.837075,-0.522928,1.365074,15.801337,-1.531202,2023-04-10
1890,SBSP3,8,9.805846,2.173509,-0.707408,2023-02-28,2022-11-09,20,0.800000,0.490292,0.727295,-0.457718,0.490292,10.543235,-1.137768,2023-03-09
249,SYNE3,2,30.145261,3.864734,-0.731707,2023-07-06,2023-04-19,30,0.766667,1.004842,1.420085,-0.359526,1.004842,15.013853,-0.979304,2023-07-26
1326,ABCB4,8,11.511404,1.944608,-1.070091,2023-01-12,2022-11-09,30,0.766667,0.383713,0.604667,-0.342276,0.383713,9.890615,-1.070091,2023-01-12
1423,DIRR3,8,16.502257,3.225806,-0.785855,2023-02-03,2022-11-28,30,0.766667,0.550075,0.814341,-0.318226,0.550075,10.516371,-0.785855,2023-02-03
1174,ELMD3,8,30.520282,6.185567,-1.002506,2023-03-02,2022-11-10,29,0.758621,1.052424,1.521604,-0.422144,1.052424,11.226561,-1.855213,2023-03-08


2 - 11:00

3 - 12:00

4 - 13:00

5 - 14:00

6 - 15:00

7 - 16:00

8 - 17:00

Comprar a abertura do horário acima

In [28]:
df_resultado

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
0,SEQL3,1,-20.229699,18.181818,-11.235955,2023-08-15,2022-09-16,30,0.366667,-0.674323,5.407876,-4.195596,-0.674323,-1.802567,-46.303661,2023-09-21
1,SEQL3,2,-32.111110,18.181818,-13.114754,2023-09-14,2023-07-06,30,0.233333,-1.070370,6.349587,-3.328618,-1.070370,-2.955120,-31.462722,2023-09-21
2,SEQL3,3,-65.824216,6.122449,-11.666667,2023-09-14,2023-07-06,30,0.233333,-2.194141,2.904539,-3.745913,-2.194141,-9.046937,-64.225976,2023-11-27
3,SEQL3,4,-27.527674,7.142857,-7.017544,2023-09-14,2023-08-02,30,0.266667,-0.917589,3.469584,-2.512925,-0.917589,-4.199334,-32.579883,2023-11-29
4,SEQL3,5,-29.174375,8.333333,-8.620690,2023-09-14,2023-08-02,30,0.200000,-0.972479,4.132326,-2.248680,-0.972479,-4.498684,-35.211827,2023-11-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1923,ARZZ3,4,-8.194888,1.329551,-2.514571,2023-10-18,2023-06-26,30,0.400000,-0.273163,0.613115,-0.864015,-0.273163,-4.713526,-9.520440,2023-11-21
1924,ARZZ3,5,-10.876690,1.307107,-2.646688,2023-08-31,2023-07-10,30,0.366667,-0.362556,0.459904,-0.838718,-0.362556,-6.237441,-12.109201,2023-11-24
1925,ARZZ3,6,-6.798253,1.604718,-2.123391,2023-10-18,2023-08-01,30,0.400000,-0.226608,0.554921,-0.747628,-0.226608,-4.260561,-8.901846,2023-11-24
1926,ARZZ3,7,-1.425851,1.257518,-0.981055,2023-10-18,2023-08-14,30,0.400000,-0.047528,0.604416,-0.482158,-0.047528,-1.202793,-4.630884,2023-11-08


In [29]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] >= 0.75) 
                    & (df_resultado['Pior Dia'] > -1.5)
                    & (df_resultado['Quantidade de Trades'] >= 20)
                    ].sort_values(['Retorno Acumulado'], ascending=False)
selecionados

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
336,DMVF3,1,40.952225,4.807692,-0.928793,2023-04-10,2022-11-30,30,0.800000,1.365074,1.837075,-0.522928,1.365074,15.801337,-1.531202,2023-04-10
1174,ELMD3,8,30.520282,6.185567,-1.002506,2023-03-02,2022-11-10,29,0.758621,1.052424,1.521604,-0.422144,1.052424,11.226561,-1.855213,2023-03-08
249,SYNE3,2,30.145261,3.864734,-0.731707,2023-07-06,2023-04-19,30,0.766667,1.004842,1.420085,-0.359526,1.004842,15.013853,-0.979304,2023-07-26
1423,DIRR3,8,16.502257,3.225806,-0.785855,2023-02-03,2022-11-28,30,0.766667,0.550075,0.814341,-0.318226,0.550075,10.516371,-0.785855,2023-02-03
1463,RAIL3,8,12.688098,2.282609,-0.962696,2022-12-13,2022-11-16,29,0.827586,0.437521,0.624352,-0.459270,0.437521,10.254318,-1.206360,2023-01-03
1326,ABCB4,8,11.511404,1.944608,-1.070091,2023-01-12,2022-11-09,30,0.766667,0.383713,0.604667,-0.342276,0.383713,9.890615,-1.070091,2023-01-12
1890,SBSP3,8,9.805846,2.173509,-0.707408,2023-02-28,2022-11-09,20,0.800000,0.490292,0.727295,-0.457718,0.490292,10.543235,-1.137768,2023-03-09
1430,IGTI11,7,8.647457,1.346046,-1.080247,2023-10-18,2023-03-29,30,0.800000,0.288249,0.513648,-0.613350,0.288249,7.482719,-1.436628,2023-10-30
1535,ITUB3,8,6.646991,1.239101,-0.340964,2022-12-08,2022-11-09,23,0.826087,0.289000,0.377568,-0.131700,0.289000,13.910910,-0.340964,2022-12-08
1052,ASAI3,8,5.127279,1.369863,-1.466276,2023-03-10,2022-11-08,21,0.761905,0.244156,0.519551,-0.637108,0.244156,6.077548,-1.466276,2023-03-10
